In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Social_Network_Ads.csv')
df

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
df2 = df.copy()
df2

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
df2 = df2.drop('User ID', axis='columns')
df2

,Gender,Age,EstimatedSalary,Purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0
...,...,...,...,...
395,Female,46,41000,1
396,Male,51,23000,1
397,Female,50,20000,1
398,Male,36,33000,0


In [5]:
unique_values = pd.unique(df2['Gender'])
print(unique_values)
i = 0
for value in unique_values:
    df2['Gender'] = df2['Gender'].replace(value, i)
    i += 1

df2

['Male' 'Female']


,Gender,Age,EstimatedSalary,Purchased
0,0,19,19000,0
1,0,35,20000,0
2,1,26,43000,0
3,1,27,57000,0
4,0,19,76000,0
...,...,...,...,...
395,1,46,41000,1
396,0,51,23000,1
397,1,50,20000,1
398,0,36,33000,0


In [6]:
X = df.loc[:, 'Age':'EstimatedSalary']
y = df['Purchased']
X

,Age,EstimatedSalary
0,19,19000
1,35,20000
2,26,43000
3,27,57000
4,19,76000
...,...,...
395,46,41000
396,51,23000
397,50,20000
398,36,33000


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

y_train = y_train.values
y_test = y_test.values


(300, 2) (300,)
(100, 2) (100,)


In [8]:
def user_fit_transform(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    a=((X - mean) / std)
    return a.to_numpy()

In [9]:
X_train = user_fit_transform(X_train)
X_test = user_fit_transform(X_test)
X_test[0:5]

array([[-0.54748976,  0.5130727 ],
       [ 0.15442019, -0.61825566],
       [-0.10879604,  0.14615539],
       [-0.54748976,  0.26846116],
       [-0.10879604, -0.61825566]])

In [10]:
from scipy.stats import mode

In [11]:
def user_confusion_matrix(y_true, y_pred):
    cm = [[0, 0], [0, 0]]
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i] == 0:
            cm[0][0] += 1
        elif y_true[i] == 0 and y_pred[i] == 1:
            cm[0][1] += 1
        elif y_true[i] == 1 and y_pred[i] == 0:
            cm[1][0] += 1
        else:
            cm[1][1] += 1
    return cm

In [12]:
def user_accuracy(y_true, y_pred):
    correct = 0
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            correct += 1
    return correct / len(y_true)

In [13]:
def Lazy_KNN(X_train,X_test,y_train,y_test):
   
    scores={}
    print('Selecting Best K Value \n \n ')
    for K in range(1,41):
    
        y_pred = []

        for x in X_test:
            points_distances = []

            for xx in X_train:
                distance = np.sqrt(np.sum((xx-x)**2))
                points_distances.append(distance)

            #choosing the k short eucledian distances
            indices = np.argpartition(points_distances, K)[:K]
            #choosing their corresponding values in target attribute
            labels = y_train[indices]
            #selecting best value for y_pred(predicting values)
            label = mode(labels)[0][0]
            y_pred.append(label)

        scores[K] = user_accuracy(y_test, y_pred)
    
    best_K = 0
    best_acc = 0
    for k in scores:
        print('For K = %d, accuracy = %.2f%%' % (k, scores[k] * 100))
        if scores[k] > best_acc:
            best_acc = scores[k]
            best_K = k    
    
    print('\n',best_K," is the best K value")
    print('\n For K = %d, accuracy = %.2f%%\n' % (best_K, scores[best_K] * 100))
    cm = user_confusion_matrix(y_test, y_pred)
    print('Confusion Matrix \n', cm)
    return y_pred

In [14]:
y_pred=Lazy_KNN(X_train,X_test,y_train,y_test)

Selecting Best K Value 
 
 
For K = 1, accuracy = 88.00%
For K = 2, accuracy = 88.00%
For K = 3, accuracy = 93.00%
For K = 4, accuracy = 93.00%
For K = 5, accuracy = 93.00%
For K = 6, accuracy = 93.00%
For K = 7, accuracy = 93.00%
For K = 8, accuracy = 93.00%
For K = 9, accuracy = 93.00%
For K = 10, accuracy = 93.00%
For K = 11, accuracy = 93.00%
For K = 12, accuracy = 93.00%
For K = 13, accuracy = 93.00%
For K = 14, accuracy = 93.00%
For K = 15, accuracy = 93.00%
For K = 16, accuracy = 93.00%
For K = 17, accuracy = 93.00%
For K = 18, accuracy = 93.00%
For K = 19, accuracy = 93.00%
For K = 20, accuracy = 93.00%
For K = 21, accuracy = 93.00%
For K = 22, accuracy = 93.00%
For K = 23, accuracy = 93.00%
For K = 24, accuracy = 93.00%
For K = 25, accuracy = 93.00%
For K = 26, accuracy = 93.00%
For K = 27, accuracy = 93.00%
For K = 28, accuracy = 93.00%
For K = 29, accuracy = 93.00%
For K = 30, accuracy = 93.00%
For K = 31, accuracy = 93.00%
For K = 32, accuracy = 93.00%
For K = 33, accuracy 

# Confusion matrix returning values

In [15]:
def user_cm_values(y_true, y_pred):
    TN=0
    FP=0
    FN=0
    TP=0
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i] == 0:
            TN+=1
        elif y_true[i] == 0 and y_pred[i] == 1:
            FP+=1
        elif y_true[i] == 1 and y_pred[i] == 0:
            FN+=1
        else:
            TP+=1
    return TN,FP,FN,TP

In [16]:
TN,FP,FN,TP=user_cm_values(y_test, y_pred)

In [17]:
print(TN,FP,FN,TP)

64 4 4 28


# Precision

In [18]:
def precision(y_true,y_pred):
    TP=0
    FP=0
    for i in range(len(y_true)):
            if y_true[i] == y_pred[i] == 1:
                TP+=1
            elif y_true[i] == 0 and y_pred[i] == 1:
                FP+=1
    pre=TP/(TP+FP)
    print('Precision: ',pre)

In [19]:
precision(y_test,y_pred)

Precision:  0.875


# Recall

In [20]:
def Recall(y_true,y_pred):
    TP=0
    FN=0
    for i in range(len(y_true)):
            if y_true[i] == y_pred[i] == 1:
                TP+=1
            elif y_true[i] == 1 and y_pred[i] == 0:
                FN+=1
    recall=TP/(TP+FP)
    print('Recall: ',recall)

In [21]:
Recall(y_test,y_pred)

Recall:  0.875


# F1-score for a binary classifier

In [22]:
def f1_score(y_true,y_pred):
    TP=0
    FN=0
    FP=0
    for i in range(len(y_true)):
            if y_true[i] == y_pred[i] == 1:
                TP+=1
            elif y_true[i] == 1 and y_pred[i] == 0:
                FN+=1
            elif y_true[i] == 0 and y_pred[i] == 1:
                FP+=1
    r=TP/(TP+FP)
    p=TP/(TP+FP)
    f1_score=(2*r*p)/(p+r)
    print('F1-Score: ',f1_score)

In [23]:
f1_score(y_test,y_pred)

F1-Score:  0.875


# Specificity

In [24]:
def Specificity(y_true,y_pred):
    TN=0
    for i in range(len(y_true)):
            if y_true[i] == y_pred[i] == 0:
                TN+=1
    x=y.value_counts()
    Spe=TN/x[0]
    print('Specificity: ',Spe)

In [25]:
Specificity(y_test,y_pred)

Specificity:  0.2490272373540856
